# Location Parse

Retrieve locations near city

*pip install requests*

In [ ]:
import requests
import json

def get_locations(lat: float, lon: float, radius: int, amenity: str) -> dict:
    overpass_query = f"""
    [out:json];
    (
      node["amenity"="{amenity}"](around:{radius},{lat},{lon});
      way["amenity"="{amenity}"](around:{radius},{lat},{lon});
    );
    out center;
    """

    url = "https://overpass-api.de/api/interpreter"

    response = requests.post(
        url,
        data=overpass_query,
        headers={"Content-Type": "text/plain"}
    )

    response.raise_for_status()
    return response.json()

def parse_lcoations(data: dict) -> list:
    locations = []
    for element in data.get("elements", []):
        if "tags" in element:
            tags = element.get("tags", {})
            name = element["tags"].get("name", "Unnamed")
            website = tags.get("website") or tags.get("contact:website")

            lat = element.get("lat") or element.get("center", {}).get("lat")
            lon = element.get("lon") or element.get("center", {}).get("lon")
            locations.append({
                "name": name,
                "latitude": lat,
                "longitude": lon,
                "website": website
            })
    return locations

LAT = 48.2931
LON = 25.9296
RADIUS = 1000  # meters
AMENITY = "restaurant"

data = get_locations(LAT, LON, RADIUS, AMENITY)
print(json.dumps(data, indent=2)[:500])
locations = parse_lcoations(data)
print(locations)

print([l for l in locations if l["website"] != None])

{
  "version": 0.6,
  "generator": "Overpass API 0.7.62.8 e802775f",
  "osm3s": {
    "timestamp_osm_base": "2026-01-18T16:05:06Z",
    "copyright": "The data included in this document is from www.openstreetmap.org. The data is made available under ODbL."
  },
  "elements": [
    {
      "type": "node",
      "id": 943053314,
      "lat": 48.2853985,
      "lon": 25.934165,
      "tags": {
        "amenity": "restaurant",
        "name": "\u0428\u0430\u0440\u043c",
        "name:uk": "\u0428\u04
[{'name': 'Шарм', 'latitude': 48.2853985, 'longitude': 25.934165, 'website': None}, {'name': 'Кнаус', 'latitude': 48.2932749, 'longitude': 25.9353508, 'website': None}, {'name': 'Квінто', 'latitude': 48.2911736, 'longitude': 25.936134, 'website': None}, {'name': 'Маестро', 'latitude': 48.28908, 'longitude': 25.9387663, 'website': 'www.maestro.cv.ua'}, {'name': 'Non Solo Pizza', 'latitude': 48.2915477, 'longitude': 25.9328578, 'website': None}, {'name': 'Ватра', 'latitude': 48.290881, 'longitude

In [2]:
from math import radians, sin, cos, sqrt, atan2, degrees

def reverse_geocode(location: dict) -> dict:
    url = "https://nominatim.openstreetmap.org/reverse"
    params = {
        "lat": location["latitude"],
        "lon": location["longitude"],
        "format": "jsonv2",
        "addressdetails": 1
    }

    headers = {
        "User-Agent": "geo-demo-app (va.melnyk@chnu.edu.ua)"
    }

    r = requests.get(url, params=params, headers=headers, timeout=10)
    r.raise_for_status()

    data = r.json()
    address = data.get("address", {})

    location["street"] = address.get("road")
    location["house_number"] = address.get("house_number")
    location["city"] = address.get("city") or address.get("town")
    location["address"] = data.get("display_name")
    return location

def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371000  # Earth radius in meters

    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)

    a = (
        sin(dlat / 2) ** 2
        + cos(radians(lat1))
        * cos(radians(lat2))
        * sin(dlon / 2) ** 2
    )

    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

def calculate_bearing(lat1, lon1, lat2, lon2):
    lat1 = radians(lat1)
    lat2 = radians(lat2)
    dlon = radians(lon2 - lon1)

    x = sin(dlon) * cos(lat2)
    y = cos(lat1) * sin(lat2) - sin(lat1) * cos(lat2) * cos(dlon)

    bearing = (degrees(atan2(x, y)) + 360) % 360
    return bearing

def bearing_to_direction(bearing):
    directions = [
        "north",
        "north-east",
        "east",
        "south-east",
        "south",
        "south-west",
        "west",
        "north-west"
    ]

    index = round(bearing / 45) % 8
    return directions[index]

location = reverse_geocode(locations[1])
location["distance"] = haversine_distance(LAT, LON, location["latitude"], location["longitude"])
bearing = calculate_bearing(LAT, LON, location["latitude"], location["longitude"])
location["direction"] = bearing_to_direction(bearing)
print(location)


{'name': 'Кнаус', 'latitude': 48.2932749, 'longitude': 25.9353508, 'website': None, 'street': 'Центральна площа', 'house_number': None, 'city': 'Чернівці', 'address': 'Кнаус, Центральна площа, Старе Місто, Чернівці, Чернівецька міська громада, Чернівецький район, Чернівецька область, 58002, Україна', 'distance': 425.8890974895354, 'direction': 'east'}


### Next 
1. Retrieve menus for given locations (?)
2. Menus for rag vectorization + pipeline 
3. 


Сорбонна
піца іжа напої

lat lon, distance <- overpass 
